In [ ]:
%pip install pandas

# Gathering data from council XLS files

In [15]:
import pandas as pd
import requests
import os

# Create list of target URLs to download
classified_urls_df = pd.read_csv('../wayback/url_classified.csv')
classified_urls_df['target_url'] = classified_urls_df['wayback_download_url'].str.split('id_/').str[1]

list_new_businesses = classified_urls_df[classified_urls_df['category'] == 'new_businesses']['target_url'].dropna().unique()
list_accounts_no_relief = classified_urls_df[classified_urls_df['category'] == 'accounts_no_relief']['target_url'].dropna().unique()
list_account_relief = classified_urls_df[classified_urls_df['category'] == 'accounts_relief']['target_url'].dropna().unique()
list_accouts_closed = classified_urls_df[classified_urls_df['category'] == 'accounts_closed']['target_url'].dropna().unique()
list_account_current_in_credit = classified_urls_df[classified_urls_df['category'] == 'accounts_current_in_credit']['target_url'].dropna().unique()

# Download raw files to local
save_dirs = ['new_businesses', 'accounts_no_relief', 'accounts_relief', 'accounts_closed', 'accounts_current_in_credit']

config = {
    'new_businesses': {
        'urls': list_new_businesses,
        'save_dir': 'new_businesses',
    },
    'accounts_no_relief': {
        'urls': list_accounts_no_relief,
        'save_dir': 'accounts_no_relief',
    },
    'accounts_relief': {
        'urls': list_account_relief,
        'save_dir': 'accounts_relief',
    },
    'accounts_closed': {
        'urls': list_accouts_closed,
        'save_dir': 'accounts_closed',
    },
    'accounts_current_in_credit': {
        'urls': list_account_current_in_credit,
        'save_dir': 'accounts_current_in_credit',
    },
}

for save_dir in save_dirs:
    os.makedirs('raw_files/' + save_dir, exist_ok=True)

for category, config in config.items():
    for url in config['urls']:
        raw_file_name = url.split('/')[-1]
        # Stream the content and write it to disk
        with requests.get(url, stream=True) as r:
            r.raise_for_status()  # raises exception if download failed
            with open('raw_files/' + config['save_dir'] + '/' + raw_file_name, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)


## Sanitization

In [17]:
import pandas as pd
from pathlib import Path

folder_path = 'raw_files/new_businesses'
column_mapping = {
    "Current Analysis Code Description": "current_analysis_code_description",
    "Primary Liable party name": "primary_liable_party_name",
    "Property Reference Number": "property_reference_number",
    "Full Property Address": "full_property_address",
    "Liable Responsibility Start Date": "liable_responsibility_start_date",
    "Current Rateable Value": "current_rateable_value",
}

all_dataframes = []
for file_path in Path(folder_path).glob('*.xlsx'):
    print(f'Processing {file_path}')
    # Load Excel file into pandas
    df = pd.read_excel(file_path, sheet_name=0)  # sheet_name=0 loads first sheet
    # Rename columns using the mapping
    df = df.rename(columns=column_mapping)
    # Add filename column for reference
    df['source_file'] = file_path.name
    # Store the dataframe
    all_dataframes.append(df)
    print(f"Successfully loaded: {file_path.name} with {len(df)} rows")

Processing raw_files/new_businesses/NNDR%20New%20Businesses%20from%20June%20to%20October%202023.xlsx
Processing raw_files/new_businesses/NNDR_New%20Businesses%20from%20October20%20to%20January21.xlsx
Processing raw_files/new_businesses/New%20Businesses%20July-Sept20_0.xlsx
Processing raw_files/new_businesses/New%20Businesses%20from%20October%202021%20to%20March%202022.xlsx
Processing raw_files/new_businesses/New%20Businesses%20from%20April22%20to%20August22.xlsx
Processing raw_files/new_businesses/NNDR_New%20Businesses%20from%20Nov23%20to%20March24.xlsx
Processing raw_files/new_businesses/New%20Businesses%20from%20July%20to%20September21.xlsx
Processing raw_files/new_businesses/NNDR%20New%20Businesses%20from%20September%202022%20to%20January%202023.xlsx
Processing raw_files/new_businesses/NNDR%20New%20Businesses_February%20to%20May2023.xlsx
Processing raw_files/new_businesses/~$New%20Businesses%20from%20April22%20to%20August22.xlsx
Processing raw_files/new_businesses/New%20businesses%2